<a href="https://colab.research.google.com/github/SaraSilvaa/FATEC_PLN_Codes/blob/master/Aula08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aula 8 - Introdução a ML para PLN

##Na aula de hoje vamos trabalhar com dois modelos clássicos: **Naives** (ou Ingênuo) e o modelo de suporte a vetor **(Support Vector Machine -SVM)**


###**Modelo de Naives:** é baseado no teorema de Bayes que consiste na probabilidade composta (um evento ocorrer apos outro evento)
####Precisa ter uma base de referencia para indicar quais são as probabilidades de um evento acontecer

* **P(C) =** Evento qualquer que pode acontecer (probabilidade simples)

* **P(C|X) =** Probabilidade de um evento acontecer depois de um evento(probabilidade composta)



###Sendo assim as probabilidades podem ser **dependentes**: onde o calculo é mais complexo ou **independentes**: onde o calculo é mais simples




#Exemplo 01 - Aplicação do modelo de Naives em um texto

In [6]:
#PASSO 1 - Modelo a ser treinado
corpus = [
    ("Eu amo PLN","positivo"),
    ("Eu odeio bugs","negativo"),
    ("Amo resolver problemas", "positivo"),
    ("Odeio erros", "negativo"),
    ("Bugs legais", "positivo"),
    ("Resolver", "positivo"),
    ("Eu não","negativo")
]

#PASSO 2 - Agora iremos pré-processar o texto
import re
from collections import defaultdict, Counter

def preprocess_text(text):
  return re.findall(r'\b\w+\b', text.lower())

processed_corpus = [(preprocess_text(text), label) for text, label in corpus]
print(processed_corpus)

#PASSO 03 Calculando as Probabilidades das frases
class_counts = Counter()  # Sara explicação: Cria um contador para contar a frequência de cada classe.
word_counts = defaultdict(Counter)  # Sara explicação: Cria um dicionário de contadores, um para cada classe.
total_words = defaultdict(int)  # Sara explicação: Cria um dicionário para contar o total de palavras por classe.

for words, label in processed_corpus:  # Sara explicação: Percorre o corpus processado, que contém palavras e seus rótulos.
  class_counts[label] += 1  # Sara explicação: Conta a frequência de cada classe no corpus.
  for word in words:  # Sara explicação: Itera sobre as palavras de cada exemplo no corpus.
    word_counts[label][word] += 1  # Sara explicação: Conta a frequência de cada palavra dentro de uma classe.
    total_words[label]  # Sara explicação: Soma o total de palavras por classe.

total_examples = sum(class_counts.values())  # Sara explicação: Calcula o total de exemplos no corpus.
prior_probabilities = {cls: count / total_examples for cls, count in class_counts.items()}  # Sara explicação: Calcula a probabilidade a priori para cada classe.

def conditional_probability(word, label, alpha=1):  # Sara explicação: Função para calcular a probabilidade condicional de uma palavra em uma classe.
  return (word_counts[label][word] + alpha) / (total_words[label] + alpha * len(word_counts[label]))  # Sara explicação: Aplica o método de suavização de Laplace para calcular a probabilidade condicional.

#PASSO 04 - Classificar um novo texto
def predict(text):  # Sara explicação: Função para prever a classe de um novo texto.
  words = preprocess_text(text)  # Sara explicação: Processa o texto (como tokenizar, remover stop words, etc.).
  probabilities = {}  # Sara explicação: Dicionário para armazenar as probabilidades de cada classe.

  for label in class_counts.keys():  # Sara explicação: Intera sobre todas as classes possíveis.
    probabilities[label] = prior_probabilities[label]  # Sara explicação: Começa com a probabilidade a priori da classe.
    for word in words:  # Sara explicação: Para cada palavra no texto, atualiza a probabilidade da classe.
      probabilities[label] *= conditional_probability(word, label)  # Sara explicação: Multiplica a probabilidade condicional de cada palavra na classe.
      return max(probabilities, key=probabilities.get), probabilities  # Sara explicação: Retorna a classe com a maior probabilidade.

#PASSO 5 - Teste com um novo texto
novo_texto = "Eu amo resolver bugs"  # Sara explicação: Define um novo texto para prever a classe.
classe,probs = predict(novo_texto)  # Sara explicação: Diz a classe do novo texto e calcula as probabilidades.

print(f'O texto "{novo_texto}"')  # Sara explicação: Imprime o texto original.
print(f'é classificado como: "{classe}"')  # Sara explicação: Exibe a classe prevista.
print(f'Probabilidades:')  # Sara explicação: Inicia a impressão das probabilidades de cada classe.
for label, prob in probs.items():  # Sara explicação: Itera sobre as probabilidades de todas as classes.
  print(f"{label}: {prob}")  # Sara explicação: Exibe a probabilidade de cada classe.


[(['eu', 'amo', 'pln'], 'positivo'), (['eu', 'odeio', 'bugs'], 'negativo'), (['amo', 'resolver', 'problemas'], 'positivo'), (['odeio', 'erros'], 'negativo'), (['bugs', 'legais'], 'positivo'), (['resolver'], 'positivo'), (['eu', 'não'], 'negativo')]
O texto "Eu amo resolver bugs"
é classificado como: "positivo"
Probabilidades:
positivo: 0.16326530612244897


#Exemplo 2 - Modelo de SVM (Support Vector Machine)
##É um modelo de aprendizagem supervisionado que faz classificação e regressão


In [21]:
# Passo 1 -  Importação das bibliotecas a serem utilizadas
from sklearn.feature_extraction.text import TfidfVectorizer  # Sara explicação: Importa a classe para converter texto em uma matriz TF-IDF.
from sklearn.svm import SVC  # Sara explicação: Importa o modelo de Classificador SVM (Máquinas de Vetores de Suporte).
from sklearn.model_selection import train_test_split  # Sara explicação: Importa a função para dividir os dados em treino e teste.
from sklearn.metrics import classification_report  # Sara explicação: Importa a função para avaliar o desempenho do modelo.

#Passo 2 - # Sara explicação: Definindo um conjunto de frases (corpus) de exemplo
corpus = [
    "Eu amo PLN", "Eu odeio bugs", "Eu amo resolver problemas",
  "Odeio erros", "Amo programação", "Não gosto de falhas"
]

classes = [ "negativo", "negativo", "positivo", "negativo", "positivo", "negativo"]


# Passo 3 -  Pre processamento e vetorização
vectorizer = TfidfVectorizer()  # Sara explicação: Cria o vetor de transformação TF-IDF para converter texto em uma matriz de características.
X = vectorizer.fit_transform(corpus)  # Sara explicação: Aplica o TF-IDF no corpus e cria a matriz de características (X).
y = classes  # Sara explicação: Define as classes (rótulos) para os dados (y).

#Passo 4 -  Dividir os dados e Treinar o modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  # Sara explicação: Divide os dados em 70% para treino e 30% para teste, mantendo a aleatoriedade.
svm_model = SVC(kernel='linear')  # Sara explicação: Cria o modelo SVM com kernel linear (para classificação linear).
svm_model.fit(X_train, y_train)  # Sara explicação: Treina o modelo SVM usando os dados de treino.

# Passo 5 - Avaliar o modelo
y_pred = svm_model.predict(X_test)  # Sara explicação: Usando o modelo treinado para fazer previsões nos dados de teste.
print(classification_report(y_test, y_pred))  # Sara explicação: Imprime o relatório de classificação com métricas de desempenho

              precision    recall  f1-score   support

    negativo       1.00      0.50      0.67         2
    positivo       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#Explicação das classificações
###**Precision (Precisão):** Mede a exatidão das previsões positivas.

###**"negativo":** 1.00 (100% de precisão para negativos).
###**"positivo":** 0.00 (nenhuma previsão correta para positivos).
###**Recall (Revocação ou Sensibilidade):** Mede a capacidade de identificar todas as instâncias positivas.

###**"negativo": 0.50** (50% de acerto para negativos).
###**"positivo": 0.00** (não encontrou nenhum positivo).
###**F1-Score (F1-Score):** Média harmônica entre precisão e recall.

###**"negativo":** 0.67 (bom desempenho para negativos).
###**"positivo":** 0.00 (não conseguiu prever positivos).
###**Support (Suporte):** Número de exemplos reais de cada classe no conjunto de teste.

###**"negativo":** 2 exemplos.
###"**positivo":** 0 exemplos (o que comprometeu o desempenho para "positivo").
###**Accuracy (Acurácia):** 50% de previsões corretas no total.

###**Macro avg (Média Macro) e Weighted avg (Média Ponderada):** A média das métricas entre as classes, ponderando ou não pelo número de exemplos

#Exemplo 03 - A comparação entre os 2 modelos

In [24]:
# 1. Importar Bibliotecas como no exemplo anterior
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Baixar o dataset de exemplo
nltk.download('movie_reviews')  # Sara explicação: Baixa o conjunto de dados de resenhas de filmes do NLTK.
from nltk.corpus import movie_reviews  # Sara explicação: Importa o corpus de resenhas de filmes.

# 2. Preparação dos dados
  # Coleta de textos e classes
documents = [(" ".join(movie_reviews.words(fileid)), category)  # Sara explicação: Coleta as palavras de cada arquivo de resenha e as associa à categoria.
              for category in movie_reviews.categories()  # Sara explicação: Para cada categoria (pos/neg), extrai os arquivos de resenhas.
              for fileid in movie_reviews.fileids(category)]  # Sara explicação: Para cada arquivo de resenha em cada categoria.

  # Separar textos e rótulos
texts, labels = zip(*documents)  # Sara explicação: Separa os textos e rótulos (classes) em duas listas (texts e labels).

  # Transformar rótulos (positivo/negativo) em 0 e 1
from sklearn.preprocessing import LabelEncoder  # Sara explicação: Importa a classe que converte rótulos categóricos em valores numéricos.
label_encoder = LabelEncoder()  # Sara explicação: Cria o objeto que fará a conversão dos rótulos.
labels = label_encoder.fit_transform(labels)  # Sara explicação: Converte os rótulos (positivo/negativo) em 0 e 1.

  # Dividir dados em treino e teste
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.3, random_state=42)  # Sara explicação: Divide os dados em 70% para treino e 30% para teste.

# 3. Representação do texto com TF-IDF
  # Criar o vetorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Sara explicação: Cria o vetorizador TF-IDF, limitando a 5000 palavras mais relevantes.

  # Ajustar e transformar os textos
X_train = vectorizer.fit_transform(texts_train)  # Sara explicação: Aplica o TF-IDF nos textos de treino.
X_test = vectorizer.transform(texts_test)  # Sara explicação: Aplica o TF-IDF nos textos de teste (sem ajustar).

# 4. Treinar os modelos
  # Treinamento do Naive Bayes
nb_model = MultinomialNB()  # Sara explicação: Cria o modelo de Naive Bayes para classificação de múltiplas classes.
nb_model.fit(X_train, labels_train)  # Sara explicação: Treina o modelo Naive Bayes com os dados de treino.
  # Predição
nb_predictions = nb_model.predict(X_test)  # Sara explicação: Faz previsões nos dados de teste usando o modelo Naive Bayes.

  # Treinamento do SVM
svm_model = SVC(kernel='linear')  # Sara explicação: Cria o modelo SVM com kernel linear (para classificação linear).
svm_model.fit(X_train, labels_train)  # Sara explicação: Treina o modelo SVM com os dados de treino.
  # Predição
svm_predictions = svm_model.predict(X_test)  # Sara explicação: Faz previsões nos dados de teste usando o modelo SVM.

# 5. Avaliação
  # Avaliação do Naive Bayes
print("****Naive Bayes Performance:*****")  # Sara explicação: Exibe o título para o desempenho do modelo Naive Bayes.
  # Limitar a 5.000 palavras mais comuns
print(classification_report(labels_test, nb_predictions, target_names=label_encoder.classes_))  # Sara explicação: Exibe o relatório de classificação do Naive Bayes, mostrando métricas como precisão, recall e F1-score.

  # Avaliação do SVM
print("*****SVM Performance:*****")  # Sara explicação: Exibe o título para o desempenho do modelo SVM.
print(classification_report(labels_test, svm_predictions, target_names=label_encoder.classes_))  # Sara explicação: Exibe o relatório de classificação do SVM.

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


****Naive Bayes Performance:*****
              precision    recall  f1-score   support

         neg       0.79      0.84      0.81       302
         pos       0.82      0.77      0.80       298

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600

*****SVM Performance:*****
              precision    recall  f1-score   support

         neg       0.82      0.80      0.81       302
         pos       0.81      0.82      0.81       298

    accuracy                           0.81       600
   macro avg       0.81      0.81      0.81       600
weighted avg       0.81      0.81      0.81       600

